In [ ]:
#STUDENT NAME: SHRUTHI
#STUDENT ID: 801218392
#HOMEWORK 3 QUESTION 1-A

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from datetime import datetime

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [2]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [3]:
from torchvision import datasets, transforms
data_path = 'C:/Users/Shruthi/Documents/HWs/RTML/hw2/data'
# Here the training set is downloaded as cifar10

cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [4]:
# The validation set is downloaded as cifar10_val
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [5]:
#The training set is defined as train_loader
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)

In [6]:
#The validation set is defined as val_loader
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)

In [7]:
model1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten(),
            nn.Linear(4096, 64),
            nn.LogSoftmax(dim=1))

In [8]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):

    correctt = 0
    totalt = 0
    
    for epoch in range (1, n_epochs+1):
        loss_train=0.0
        for imgs, labels in train_loader:
            outputs= model(imgs)
            loss=loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train+=loss.item()
            
        if epoch ==1 or epoch % 10==0:
            now = datetime.now()
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
            print(dt_string, "Epoch: %d, Loss: %f" % (epoch, float(loss)))
            
        with torch.no_grad():
            for imgs, labels in train_loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
            totalt += labels.shape[0]
            correctt += int((predicted == labels).sum())
            acctt = float(correctt/totalt)
    print("Accuracy in Training: ",acctt)

In [9]:
optimizer = optim.SGD(model1.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 300,
    optimizer = optimizer,
    model = model1,
    loss_fn = loss_fn,
    train_loader = train_loader)

23/03/2022 20:40:56 Epoch: 1, Loss: 2.135571
23/03/2022 20:45:09 Epoch: 10, Loss: 1.198534
23/03/2022 20:49:54 Epoch: 20, Loss: 0.981306
23/03/2022 20:54:38 Epoch: 30, Loss: 0.818489
23/03/2022 20:59:05 Epoch: 40, Loss: 0.725313
23/03/2022 21:03:30 Epoch: 50, Loss: 0.663042
23/03/2022 21:08:03 Epoch: 60, Loss: 0.619448
23/03/2022 21:12:40 Epoch: 70, Loss: 0.587649
23/03/2022 21:17:08 Epoch: 80, Loss: 0.561599
23/03/2022 21:21:47 Epoch: 90, Loss: 0.531873
23/03/2022 21:26:25 Epoch: 100, Loss: 0.504572
23/03/2022 21:30:56 Epoch: 110, Loss: 0.481386
23/03/2022 21:35:40 Epoch: 120, Loss: 0.458986
23/03/2022 21:40:26 Epoch: 130, Loss: 0.440421
23/03/2022 21:45:16 Epoch: 140, Loss: 0.420034
23/03/2022 21:50:14 Epoch: 150, Loss: 0.402287
23/03/2022 21:55:15 Epoch: 160, Loss: 0.383797
23/03/2022 22:00:12 Epoch: 170, Loss: 0.370682
23/03/2022 22:05:02 Epoch: 180, Loss: 0.359127
23/03/2022 22:10:03 Epoch: 190, Loss: 0.344010
23/03/2022 22:15:03 Epoch: 200, Loss: 0.333038
23/03/2022 22:19:56 Epoc

In [10]:
def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correctv = 0
        totalv = 0
        
        with torch.no_grad():
            for imgs, labels in val_loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
            totalv += labels.shape[0]
            correctv += int((predicted == labels).sum())
            acctv = float(correctv/totalv)
    print("Accuracy in Validation: ",acctv)
validate (model1, train_loader, val_loader)

Accuracy in Validation:  0.6875


In [12]:
import torchvision.models as models
from ptflops import get_model_complexity_info

net = model1
macs, params = get_model_complexity_info(net, (3, 32, 32), as_strings=True,
                                           print_per_layer_stat=True, verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Sequential(
  0.263 M, 100.000% Params, 0.001 GMac, 100.000% MACs, 
  (0): Conv2d(0.0 M, 0.171% Params, 0.0 GMac, 60.864% MACs, 3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(0.0 M, 0.000% Params, 0.0 GMac, 2.174% MACs, )
  (2): MaxPool2d(0.0 M, 0.000% Params, 0.0 GMac, 2.174% MACs, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Flatten(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, start_dim=1, end_dim=-1)
  (4): Linear(0.262 M, 99.829% Params, 0.0 GMac, 34.788% MACs, in_features=4096, out_features=64, bias=True)
  (5): LogSoftmax(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, dim=1)
)
Computational complexity:       0.0 GMac
Number of parameters:           262.66 k


In [13]:
numel_list = [p.numel() for p in model1.parameters()]
sum(numel_list), numel_list

(262656, [432, 16, 262144, 64])